In [1]:
using LinearAlgebra

In [4]:
vec_list = [normalize!(randn(6)) for j=1:4]


4-element Vector{Vector{Float64}}:
 [0.37883268169182394, 0.45916046428959695, 0.33180777202964445, 0.6806612235142964, -0.1650289262247398, -0.21219524511495805]
 [-0.2935795789633731, -0.08386199559223018, 0.20572809444195458, 0.8961875343613418, 0.17800160085130265, 0.1720973010188473]
 [-0.4446410737575766, 0.3552153895079222, 0.32154402150243866, 0.5611802758554121, 0.39396953712961513, -0.3202974970635955]
 [0.21114583018874325, -0.1953671323409361, -0.7110864111174259, -0.49169131556724677, -0.15251850020673613, 0.38286158753440197]

In [5]:

mat = ones((4,4))-Matrix(I,(4,4))

4×4 Matrix{Float64}:
 0.0  1.0  1.0  1.0
 1.0  0.0  1.0  1.0
 1.0  1.0  0.0  1.0
 1.0  1.0  1.0  0.0

In [6]:
println(cond(mat))

3.0000000000000004


In [16]:
for m=2:20

    mat = -ones((m,m))+Matrix(I,(m,m))
    
    #println(cond(mat))
    
    F = eigen(mat)
    
    display(F.values)
    
    display(F.vectors)
end

2-element Vector{Float64}:
 -1.0
  1.0

2×2 Matrix{Float64}:
 -0.707107  -0.707107
 -0.707107   0.707107

3-element Vector{Float64}:
 -1.9999999999999982
  0.9999999999999999
  1.0000000000000002

3×3 Matrix{Float64}:
 -0.57735   0.707107  -0.408248
 -0.57735  -0.707107  -0.408248
 -0.57735   0.0        0.816497

4-element Vector{Float64}:
 -2.9999999999999982
  0.9999999999999996
  1.0
  1.0

...

18×18 Matrix{Float64}:
 -0.235702  -0.112623      0.136517     …  -0.0383482    -0.0606339
 -0.235702  -0.112623      0.136517        -0.0383482    -0.0606339
 -0.235702  -0.112623      0.136517        -0.0383482    -0.0606339
 -0.235702  -0.112623      0.136517        -0.0383482    -0.0606339
 -0.235702  -0.68987      -0.274923        -0.115045     -0.0606339
 -0.235702   0.401031      0.625041     …  -0.115045     -0.0606339
 -0.235702  -0.32535       0.394376        -0.115045     -0.0606339
 -0.235702   0.142446     -0.172667        -0.115045     -0.0606339
 -0.235702   0.2323       -0.281585        -0.0383482    -0.0606339
 -0.235702   0.129681     -0.157194        -0.0383482    -0.0606339
 -0.235702   0.240574     -0.291614     …  -0.0383482    -0.0606339
 -0.235702   0.0221793    -0.0268849       -0.0383482    -0.0606339
 -0.235702   0.118257     -0.143347        -0.0383482    -0.0606339
 -0.235702   0.204644     -0.248061        -0.0383482    -0.0606339
 -0.235702   0.0101089   

19-element Vector{Float64}:
 -17.999999999999996
   0.9999999999999994
   0.9999999999999994
   0.9999999999999994
   0.9999999999999996
   0.9999999999999996
   0.9999999999999996
   0.9999999999999999
   0.9999999999999999
   0.9999999999999999
   0.9999999999999999
   0.9999999999999999
   0.9999999999999999
   1.0
   1.0000000000000002
   1.0000000000000002
   1.0000000000000002
   1.0000000000000002
   1.0000000000000029

19×19 Matrix{Float64}:
 -0.229416  -0.123006     0.242607   …  -0.0360457   -0.0540738  -0.129677
 -0.229416  -0.123006     0.242607      -0.0360457   -0.0540738  -0.129677
 -0.229416  -0.123006     0.242607      -0.0360457   -0.0540738  -0.129677
 -0.229416  -0.123006     0.242607      -0.0360457   -0.0540738  -0.129677
 -0.229416   0.184709     0.0257288     -0.0898137   -0.0540738  -0.129677
 -0.229416  -0.518326     0.0257288  …  -0.0898137   -0.0540738  -0.129677
 -0.229416   0.259748     0.0257288     -0.0898137   -0.0540738  -0.129677
 -0.229416   0.718977     0.0257288     -0.0898137   -0.0540738  -0.129677
 -0.229416   0.0972915   -0.18246       -0.00544195  -0.0540738   0.233959
 -0.229416  -0.0124051   -0.18246       -0.00544195  -0.0540738   0.233959
 -0.229416  -0.0210506   -0.18246    …  -0.00544195  -0.0540738   0.233959
 -0.229416  -0.031195    -0.18246       -0.00544195  -0.0540738   0.233959
 -0.229416   0.0167614   -0.116932      -0.254757    -0.0540738  -0.129677
 -

20-element Vector{Float64}:
 -18.999999999999996
   0.9999999999999993
   0.9999999999999993
   0.9999999999999996
   0.9999999999999997
   0.9999999999999998
   0.9999999999999998
   0.9999999999999998
   0.9999999999999998
   0.9999999999999998
   0.9999999999999998
   0.9999999999999999
   1.0000000000000002
   1.0000000000000002
   1.0000000000000002
   1.0000000000000002
   1.0000000000000002
   1.0000000000000002
   1.000000000000001
   1.0000000000000042

20×20 Matrix{Float64}:
 -0.223607   0.0104344     0.0153619    …   0.0816307    -0.0660245
 -0.223607   0.0104344     0.0153619        0.0816307    -0.0660245
 -0.223607   0.0104344     0.0153619        0.0816307    -0.0660245
 -0.223607   0.0104344     0.0153619        0.0816307    -0.0660245
 -0.223607  -0.054504     -0.0177331       -0.103773     -0.0660245
 -0.223607  -0.054504     -0.0177331    …  -0.103773     -0.0660245
 -0.223607  -0.054504     -0.0177331       -0.103773     -0.0660245
 -0.223607   0.918739     -0.0177331       -0.103773     -0.0660245
 -0.223607  -0.166452      0.883857        -0.103773     -0.0660245
 -0.223607  -0.16954      -0.202374        -0.103773     -0.0660245
 -0.223607  -0.0613372    -0.0996629    …  -0.103773     -0.0660245
 -0.223607   0.0184031     0.0101207       -0.103773     -0.0660245
 -0.223607  -0.235264     -0.334691        -0.103773     -0.0660245
 -0.223607  -0.169439     -0.222274        -0.103773     -0.0660245
 -0.223607  -0.0336107   